<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=f9d51496196279afce3d9885c609775922f82741efcd0dca9fa15f8859a369d3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-17 15:11:12
-------------------
qualified stocks: 88
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  2.18 L
Current:  1.43 C
-------------------
Today PnL: 92.78 K (0.66%)
Current PnL: -16.62 L (-11.18%)
CY Booked + Current PnL: -5.59 L (-3.76%)
-------------------
Total profit:  2.31 L
Total loss:  -18.93 L
-------------------
Total Booked + Current PnL: 21.39 L (17.58%)
Total Booked PnL: 38.01 L (31.23%)
Curr Year Booked PnL: 11.03 L (7.74%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 82.51 L (57.87%)
Deployed:  1.22 C
Current:  1.43 C
CAGR/XIRR %: 9.83%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SBIN,863.00,-8.48,77.0,M-LC,4.42,227171.0,25692.0,1522.0,3.94,12.75,0.67,13.51,60.0,16.88,1.62,26.95,XY25,NTT,BANKS
5,APOLLOHOSP,8285.00,-13.09,59.0,H-LC,11.07,173569.0,17392.0,8696.0,0.93,11.14,5.01,16.71,22.0,2.00,1.24,30.52,X40N,BTT,HEALTHCARE
78,TTKPRESTIG,770.00,105.55,61.0,M-SC,4.41,88576.0,-12201.0,12294.0,0.60,-12.11,13.88,0.09,245.0,-0.99,0.63,16.37,OX40N,NTT,DURABLES
50,NATIONALUM,247.44,-43.66,65.0,H-MC,5.62,113091.0,12587.0,18049.0,-1.57,12.52,15.96,30.49,79.0,0.70,0.81,48.99,MH,ATH,METALS
57,RELIANCE,1533.00,-12.91,54.0,H-LC,5.86,218291.0,7925.0,20847.0,0.31,3.77,9.55,13.68,37.0,0.38,1.55,20.91,XY25,NTT,REFINERIES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,IEX,219.00,-29.69,55.0,H-SC,17.30,201107.0,1153.0,96069.0,0.67,0.58,47.77,48.63,115.0,0.01,1.43,12.20,XR,NTT,MISC
35,ICICIPRULI,790.00,-21.22,42.0,H-MC,2.24,135555.0,-233.0,42984.0,-0.89,-0.17,31.71,31.48,107.0,-0.01,0.97,11.93,X40,ATH,INSURANCE
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.79,248695.0,964.0,74086.0,-0.06,0.39,29.79,30.30,92.0,0.01,1.77,13.73,X40,ATH,ELECTRICAL
52,PGHH,17973.08,-30.68,53.0,H-MC,4.37,200700.0,-120.0,68900.0,0.99,-0.06,34.33,34.25,80.0,-0.00,1.43,5.13,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VBL,671.64,-16.58,38.0,H-LC,5.60,300498.0,-15344.0,128012.0,-0.14,-4.86,42.60,35.67,5.0,-0.12,2.14,8.32,X40N,ATH,FMCG
6,ASIANPAINT,3465.66,-13.06,46.0,H-LC,8.17,219331.0,-32437.0,85649.0,-0.41,-12.88,39.05,21.13,27.0,-0.38,1.56,17.98,X40,ATH,PAINTS
42,ITC,452.00,-37.92,48.0,H-LC,2.91,200001.0,-137.0,21020.0,-0.88,-0.07,10.51,10.44,4.0,-0.01,1.42,5.49,X40,NTT,FMCG
44,JIOFIN,387.00,-0.10,50.0,H-LC,13.39,271502.0,4137.0,61305.0,0.11,1.55,22.58,24.48,48.0,0.07,1.93,57.37,XY24,BTT,FINANCE
20,CIPLA,1795.00,-19.78,51.0,H-LC,5.82,213720.0,9220.0,32186.0,0.76,4.51,15.06,20.25,10.0,0.29,1.52,13.64,X40N,BTT,PHARMA


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.20,-1.45,65.0,H-LC,0.99,244125.0,-2026.0,81245.0,1.56,-0.82,33.28,32.18,16.0,-0.02,1.74,34.92,X200,ATH,IT
42,ITC,452.00,-37.92,48.0,H-LC,2.91,200001.0,-137.0,21020.0,-0.88,-0.07,10.51,10.44,4.0,-0.01,1.42,5.49,X40,NTT,FMCG
51,NESTLEIND,1377.00,-8.48,57.0,H-LC,4.13,281806.0,16380.0,40411.0,-0.68,6.17,14.34,21.40,11.0,0.41,2.01,13.70,XY25,NTT,FMCG
84,VBL,671.64,-16.58,38.0,H-LC,5.60,300498.0,-15344.0,128012.0,-0.14,-4.86,42.60,35.67,5.0,-0.12,2.14,8.32,X40N,ATH,FMCG
20,CIPLA,1795.00,-19.78,51.0,H-LC,5.82,213720.0,9220.0,32186.0,0.76,4.51,15.06,20.25,10.0,0.29,1.52,13.64,X40N,BTT,PHARMA


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VAIBHAVGBL,521.00,64.74,70.0,H-SC,8.49,142720.0,-40055.0,151126.0,6.33,-21.92,105.89,60.77,125.0,-0.27,1.02,27.83,XR,NTT,JEWELLERY
70,SURYODAY,240.00,46.93,62.0,H-SC,2.80,135629.0,-43442.0,109168.0,4.74,-24.26,80.49,36.71,135.0,-0.40,0.97,34.57,XR,NTT,BANKS
17,CAMPUS,393.00,-22.86,65.0,M-SC,1.45,160742.0,-7033.0,62882.0,4.13,-4.19,39.12,33.28,210.0,-0.11,1.14,24.59,XY24,NTT,FOOTWEAR
63,SBIN,863.00,-8.48,77.0,M-LC,4.42,227171.0,25692.0,1522.0,3.94,12.75,0.67,13.51,60.0,16.88,1.62,26.95,XY25,NTT,BANKS
18,CAMS,5211.76,-1.17,62.0,H-SC,4.34,112252.0,10248.0,33676.0,3.86,10.05,30.00,43.06,122.0,0.30,0.80,28.88,X40N,ATH,MISC


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SFL,1287.00,36.64,45.0,M-SC,8.44,180342.0,-81876.0,151704.0,-4.07,-31.22,84.12,26.63,229.0,-0.54,1.28,12.13,XY24,NTT,MISC
37,INDIAMART,4850.92,-51.65,45.0,H-SC,12.09,135176.0,11840.0,121915.0,-2.25,9.60,90.19,108.45,119.0,0.10,0.96,34.10,AR,ATH,MISC
65,SHALBY,327.00,1061.08,64.0,M-SC,2.55,157935.0,-23208.0,68670.0,-2.08,-12.81,43.48,25.10,235.0,-0.34,1.12,26.57,XY24,NTT,HEALTHCARE
79,UJJIVANSFB,60.00,96.03,55.0,H-SC,12.41,123660.0,-18819.0,38335.0,-1.97,-13.21,31.00,13.70,163.0,-0.49,0.88,45.95,OX40N,NTT,BANKS
14,BERGEPAINT,680.00,-19.60,45.0,H-MC,6.00,216270.0,-11069.0,59128.0,-1.88,-4.87,27.34,21.14,106.0,-0.19,1.54,21.90,XY24,NTT,PAINTS


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.0,105.55,61.0,M-SC,4.41,88576.0,-12201.0,12294.0,0.60,-12.11,13.88,0.09,245.0,-0.99,0.63,16.37,OX40N,NTT,DURABLES
79,UJJIVANSFB,60.0,96.03,55.0,H-SC,12.41,123660.0,-18819.0,38335.0,-1.97,-13.21,31.00,13.70,163.0,-0.49,0.88,45.95,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.42,61.0,H-SC,1.70,224820.0,-44847.0,81183.0,1.48,-16.63,36.11,13.47,138.0,-0.55,1.60,13.71,XY24,NTT,PAINTS
19,CERA,9475.0,-20.29,44.0,H-SC,2.16,145337.0,-30566.0,72581.0,-0.05,-17.38,49.94,23.89,149.0,-0.42,1.04,25.52,OX40N,NTT,CERAMICS
73,TATAELXSI,9161.0,-20.18,52.0,H-MC,7.82,107986.0,-23849.0,66077.0,-0.03,-18.09,61.19,32.03,98.0,-0.36,0.77,20.95,OX40N,NTT,IT


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
75,TCS,4476.75,-25.15,59.0,H-LC,12.76,298469.0,-47487.0,122342.0,2.03,-13.73,40.99,21.64,1.0,-0.39,2.13,5.73,X40,ATH,IT
40,INFY,2275.00,-16.73,55.0,H-LC,8.76,324463.0,11575.0,160122.0,0.99,3.70,49.35,54.87,3.0,0.07,2.31,10.59,X40,BTT,IT
42,ITC,452.00,-37.92,48.0,H-LC,2.91,200001.0,-137.0,21020.0,-0.88,-0.07,10.51,10.44,4.0,-0.01,1.42,5.49,X40,NTT,FMCG
84,VBL,671.64,-16.58,38.0,H-LC,5.60,300498.0,-15344.0,128012.0,-0.14,-4.86,42.60,35.67,5.0,-0.12,2.14,8.32,X40N,ATH,FMCG
1,ABB,7934.00,-38.11,65.0,H-LC,11.09,258816.0,-2803.0,122006.0,1.00,-1.07,47.14,45.57,7.0,-0.02,1.84,10.13,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,65.0,M-MC,6.97,228716.0,3754.0,163898.0,0.02,1.67,71.66,74.53,192.0,0.02,1.63,34.46,XY24,BTT,STEEL
15,BLUESTARCO,2326.38,12.23,61.0,H-MC,3.72,193800.0,29130.0,38838.0,2.59,17.69,20.04,41.28,89.0,0.75,1.38,27.08,X40N,ATH,AC
85,VOLTAS,1918.49,0.88,58.0,H-MC,2.69,211725.0,19983.0,76052.0,-0.30,10.42,35.92,50.08,99.0,0.26,1.51,17.93,XY25,ATH,AC
30,HAVELLS,2069.16,0.34,62.0,H-MC,1.79,248695.0,964.0,74086.0,-0.06,0.39,29.79,30.30,92.0,0.01,1.77,13.73,X40,ATH,ELECTRICAL
18,CAMS,5211.76,-1.17,62.0,H-SC,4.34,112252.0,10248.0,33676.0,3.86,10.05,30.00,43.06,122.0,0.30,0.80,28.88,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,247.44,-43.66,65.0,H-MC,5.62,113091.0,12587.0,18049.0,-1.57,12.52,15.96,30.49,79.0,0.70,0.81,48.99,MH,ATH,METALS
86,WHIRLPOOL,2270.00,-40.41,53.0,M-SC,0.68,100688.0,9190.0,69565.0,-0.08,10.04,69.09,86.07,223.0,0.13,0.72,46.22,XR,NTT,DURABLES
37,INDIAMART,4850.92,-51.65,45.0,H-SC,12.09,135176.0,11840.0,121915.0,-2.25,9.60,90.19,108.45,119.0,0.10,0.96,34.10,AR,ATH,MISC
87,WIPRO,420.00,-10.16,56.0,M-LC,5.56,157548.0,6603.0,102847.0,1.13,4.37,65.28,72.51,53.0,0.06,1.12,10.66,XR,NTT,IT
12,BANKINDIA,190.00,-29.19,66.0,H-MC,5.91,183883.0,4075.0,108344.0,1.33,2.27,58.92,62.52,88.0,0.04,1.31,37.13,XR,NTT,BANKS


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-41.65,53.0,L-SC,41.77,85248.0,-28301.0,68352.0,-0.05,-24.92,80.18,35.27,268.0,-0.41,0.61,108.65,XR,NTT,HOTELS
3,ALKYLAMINE,4546.37,-2.99,51.0,H-SC,13.37,90300.0,-10663.0,105190.0,1.87,-10.56,116.49,93.63,148.0,-0.10,0.64,34.50,SR,ATH,CHEMICALS
49,MASFIN,398.61,-16.44,54.0,H-SC,12.90,94935.0,-3045.0,24645.0,-1.08,-3.11,25.96,22.05,152.0,-0.12,0.68,37.99,XR,ATH,FINANCE
50,NATIONALUM,247.44,-43.66,65.0,H-MC,5.62,113091.0,12587.0,18049.0,-1.57,12.52,15.96,30.49,79.0,0.70,0.81,48.99,MH,ATH,METALS
8,ATULAUTO,844.00,3872.97,64.0,M-SC,6.47,123888.0,-23480.0,75299.0,1.37,-15.93,60.78,35.16,236.0,-0.31,0.88,26.54,XY24,NTT,AUTO


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,ATULAUTO,844.0,3872.97,64.0,M-SC,6.47,123888.0,-23480.0,75299.0,1.37,-15.93,60.78,35.16,236.0,-0.31,0.88,26.54,XY24,NTT,AUTO
22,COFFEEDAY,80.0,-41.65,53.0,L-SC,41.77,85248.0,-28301.0,68352.0,-0.05,-24.92,80.18,35.27,268.0,-0.41,0.61,108.65,XR,NTT,HOTELS
27,GREENPANEL,537.0,238.50,67.0,M-SC,4.08,151012.0,-32066.0,111039.0,0.36,-17.51,73.53,43.14,230.0,-0.29,1.08,42.05,XY24,NTT,MISC
54,RAJESHEXPO,518.0,1889.90,67.0,L-SC,4.42,54301.0,-82876.0,82966.0,1.94,-60.42,152.79,0.07,267.0,-1.00,0.39,35.05,OX40N,NTT,JEWELLERY
76,TITAGARH,1548.0,0.53,68.0,H-SC,4.12,164850.0,-27165.0,106048.0,-0.66,-14.15,64.33,41.08,158.0,-0.26,1.17,39.69,XY24,NTT,ENGINEERING


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.49
1,25,42.77
2,50,73.19


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.33
LC    34.06
MC    22.63
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.45
X40      14.99
X40N     12.86
XR       11.44
XY25      9.62
AR        8.52
OX40N     8.03
X200      1.74
MH        1.68
X5K       1.43
SR        1.26
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    28.50
H-LC    25.87
H-MC    19.73
M-SC    13.31
M-LC     7.14
M-MC     2.60
L-SC     1.52
L-LC     1.05
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
IT,13.23,-13.20,74.93
FMCG,11.55,-4.35,40.87
FINANCE,9.48,-15.64,60.93
BANKS,7.76,-10.20,60.94
MISC,7.32,-11.83,74.02
PAINTS,5.70,-15.32,32.51
ELECTRICAL,5.59,-4.76,62.50
AC,3.60,4.04,28.46
INSURANCE,3.35,-7.12,43.86


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2848671.0,22
XR,1305424.0,14
AR,1300369.0,9
X40,745220.0,10
X40N,608117.0,10
OX40N,521354.0,10
XY25,400347.0,7
SR,243431.0,2
X5K,117539.0,1


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3411984.0,29
M-SC,1378135.0,17
H-LC,1240949.0,15
H-MC,1184728.0,15
M-LC,376928.0,5
M-MC,313539.0,2
L-SC,248212.0,3
L-MC,60490.0,1
L-LC,36316.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1029389.0      6
M-SC       XY24       797139.0      7
H-SC       AR         784852.0      5
           XR         754347.0      7
H-MC       XY24       492279.0      4
H-LC       X40        461373.0      5
           AR         354147.0      2
M-MC       XY24       313539.0      2
H-SC       X40N       287815.0      4
           OX40N      250635.0      4
           SR         243431.0      2
H-LC       X40N       221621.0      4
H-MC       X40        214585.0      4
           XY25       186713.0      2
L-SC       XR         165246.0      2
M-SC       AR         161370.0      2
M-LC       XY24       155020.0      2
M-SC       OX40N      121676.0      4
M-LC       X5K        117539.0      1
M-SC       XY25       114538.0      1
           XR         114150.0      2
H-MC       XR         108344.0      1
M-LC       XR         102847.0      1
H-MC       X40N        98681.0      2
L-SC       OX40N       82966.0      1
H-LC       X200        81245.0      1
M-SC       X40         69262.0      1
H-MC       OX40N       66077.0      1
H-SC       MH          61515.0      1
H-LC       XY24        61305.0      1
           XY25        61258.0      2
L-MC       XR          60490.0      1
L-LC       XY25        36316.0      1
H-MC       MH          18049.0      1
M-LC       XY25         1522.0      1

In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 50.0 seconds
